In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
import warnings
from openpyxl.styles import Alignment 
from functions import process_exports, aggregate_exports
warnings.filterwarnings('ignore')

###

season_end_dates = pd.read_excel("FM Quick Data.xlsx", sheet_name='Countries')
folder_import = "rawdata/"
folder_export = "data/"

###

date = '30/05/2023'
club = 'Newcastle'
csv_file = "JUN23" 
xlsx_file = csv_file

In [2]:
fm_data = pd.read_html(os.path.join(folder_import, 'stats.html'), encoding="UTF-8", thousands=".", decimal=",")[0]
att_data = pd.read_html(os.path.join(folder_import, "atts.html"), encoding="UTF-8", thousands=".", decimal=",")[0]
genie_data = pd.read_csv(os.path.join(folder_import, "genie.csv"), encoding="latin1", sep=';')
interest_data = pd.read_html(os.path.join(folder_import, "int.html"), encoding="UTF-8", thousands=".", decimal=",")[0]
fm_data['Date'] = datetime.strptime(date, '%d/%m/%Y')

print("FM Stats\n", fm_data.shape)
print("\nFM Attributes\n", att_data.shape)
print("\nGenie\n", genie_data.shape)
print("\nInterested\n", interest_data.shape)

FM Stats
 (4794, 58)

FM Attributes
 (4794, 59)

Genie
 (4794, 34)

Interested
 (4114, 7)


In [4]:
main, stats = process_exports(fm_data, att_data, genie_data, interest_data, season_end_dates, club, folder_export, csv_file)

Full list of players exported.
File JUN23.csv created at "data/"


## Excel

### Filter settings

In [12]:
my_club = "Newcastle"
n_best = 15
val = float('inf')
age = 30
mins = 500
wage = float('inf')
interest = 1

### HEATMAP - Top N players by position

In [17]:
pd.set_option('display.max_columns', 50)
excel_file = "Heatmap_TopN_" + csv_file + ".xlsx"
excel_file_path = os.path.join(folder_export, excel_file)

positions = ['GK', 'LB', 'CB', 'RB', 'DM', 'CM', 'AM', 'RW', 'LW', 'ST']
nth = stats.copy()
nth = nth[(nth['Mins'] >= mins) &
          (nth['Club'] != my_club) &
          (nth['Value'] <= val) &
          (nth['Wage'] < wage) &
          (nth['Age'] <= age)]

nth['GK%_rk'] = nth.groupby(['P1'])['GK%'].rank(ascending=False)
nth['DFa%_rk'] = nth.groupby(['P1'])['DFa%'].rank(ascending=False)
nth['DFg%_rk'] = nth.groupby(['P1'])['DFg%'].rank(ascending=False)
nth['PAS%_rk'] = nth.groupby(['P1'])['PAS%'].rank(ascending=False)
nth['DRB%_rk'] = nth.groupby(['P1'])['DRB%'].rank(ascending=False)
nth['ST%_rk'] = nth.groupby(['P1'])['ST%'].rank(ascending=False)
nth['AvR_rk'] = nth.groupby(['P1'])['Av_Rat'].rank(ascending=False)

nth['Aux1'] = nth['P1'] + nth['GK%_rk'].astype(str)
nth['Aux2'] = nth['P1'] + nth['DFa%_rk'].astype(str)
nth['Aux3'] = nth['P1'] + nth['DFg%_rk'].astype(str)
nth['Aux4'] = nth['P1'] + nth['PAS%_rk'].astype(str)
nth['Aux5'] = nth['P1'] + nth['DRB%_rk'].astype(str)
nth['Aux6'] = nth['P1'] + nth['ST%_rk'].astype(str)
nth['Aux7'] = nth['P1'] + nth['AvR_rk'].astype(str)

# Create an empty DataFrame to store the results
scouting = pd.DataFrame()

for pos in positions:
    # Apply your filtering conditions one by one
    filtered_df = nth[(nth['P1'] == pos)]
    
    if pos == "GK":
        filtered_df = filtered_df[
            (filtered_df['GK%_rk'] < (n_best * 2) + 1) |
            (filtered_df['AvR_rk'] < (n_best * 2) + 1)]
    elif pos in ["RB", "LB"]:
        filtered_df = filtered_df[
            (filtered_df['DFg%_rk'] < n_best + 1) |
            (filtered_df['PAS%_rk'] < n_best + 1) |
            (filtered_df['DRB%_rk'] < n_best + 1) |
            (filtered_df['AvR_rk'] < n_best + 1)
        ]
    elif pos == "CB":
        filtered_df = filtered_df[
            (filtered_df['DFa%_rk'] < n_best + 1) |
            (filtered_df['PAS%_rk'] < n_best + 1) |
            (filtered_df['DRB%_rk'] < n_best + 1) |
            (filtered_df['AvR_rk'] < n_best + 1)
        ]
    elif pos in ["DM", "CM"]:
        filtered_df = filtered_df[
            (filtered_df['DFg%_rk'] < n_best + 1) |
            (filtered_df['PAS%_rk'] < n_best + 1) |
            (filtered_df['DRB%_rk'] < n_best + 1) |
            (filtered_df['ST%_rk'] < n_best + 1) |
            (filtered_df['AvR_rk'] < n_best + 1)
        ]
    elif pos in ["RW", "LW", "AM", "ST"]:
        filtered_df = filtered_df[
            (filtered_df['PAS%_rk'] < n_best + 1) |
            (filtered_df['DRB%_rk'] < n_best + 1) |
            (filtered_df['ST%_rk'] < n_best + 1) |
            (filtered_df['AvR_rk'] < n_best + 1)
        ]
    
    # Concatenate the filtered DataFrame with the result DataFrame
    scouting = pd.concat([scouting, filtered_df])

scouting.reset_index(drop=True, inplace=True)
scouting = scouting.copy()
heatmap_top = scouting.style.background_gradient(cmap='RdYlGn', vmin=0).set_properties(**{'font-size': '14px'}).format(precision=2)

with pd.ExcelWriter(excel_file_path, engine='openpyxl') as excel_writer:
    heatmap_top.to_excel(excel_writer, sheet_name='Sheet1', index=False)
    workbook = excel_writer.book
    worksheet = excel_writer.sheets['Sheet1']

    for i, column in enumerate(scouting.columns):
        max_length = max(scouting[column].astype(str).apply(len).max(), len(column))
        worksheet.column_dimensions[worksheet.cell(row=1, column=i + 1).column_letter].width = max_length + 2

    cell_alignment = Alignment(horizontal='center')  # Center-align the entire column
    for i, column in enumerate(scouting.columns):
        if column not in ["Position", "Name", "Club", "Value", "Wage", "Personality"]:
            column_index = i + 1  # Get the numerical column index
            for row in range(2, len(scouting) + 2):  # Start from row 2 (data rows)
                cell = worksheet.cell(row=row, column=column_index)
                cell.alignment = cell_alignment

print(f'DataFrame has been exported to {excel_file}')

DataFrame has been exported to Heatmap_TopN_JUN23.xlsx


### Filtered list of players

In [20]:
pd.set_option('display.max_columns', 50)
excel_file = "Players_Filtered_" + csv_file + ".xlsx"
excel_file_path = os.path.join(folder_export, excel_file)
excel_table = stats

excel_column_order = [
    'Rgn', 'Half', 'Date', 'UID', 'EU', 'Nat', 'Interested', 'P1', 'P2', 'Rt1', 'Pt1', 'RtG', 'PtG', 'Position', 'Age',
    'Name', 'Av_Rat', 'Club', 'Division', 'Based', 'Apps', 'Starts', 'Mins', 'GK%', 'DFg%', 'DFa%', 'xGP/90', 'CS/90', 'Gls_Conc/90',
    'Blocks/90', 'Intcp/90', 'Clear/90', 'PAS%', 'OP-KP/90', 'CCC/90', 'Pr_Pas/90', 'xA/90', 'Pas_C/90', 'DRB%', 'ST%', 'Drb/90',
    'Cr_C/90', 'NP-xG/90', 'Gls/90', 'NP-xG/Shot', 'Value', 'Wage', 'Expires', 'Det', 'Prof', 'Amb', 'Acc', 'Pac', 'Height', 'Personality']

excel_table = excel_table[excel_column_order]

excel_table = excel_table[
    #(excel_table['Interest'] == 1) &
    (excel_table['Mins'] >= mins) &
    (excel_table['Value'] <= val) &
    (excel_table['Age'] <= age)]

excel_table.to_excel(excel_file_path, index=False)
print(f'DataFrame has been exported to {excel_file}')

DataFrame has been exported to Players_Filtered_JUN23.xlsx
